In [1]:
import pandas as pd

In [15]:
df = pd.read_csv('DecisionTree.csv')
df = df.drop("ID",axis=1)
training_data = df.values
header = ["Age","Income","Gender","Marital Status","Buys"]


array([['<21', 'High', 'Male', 'Single', 'No'],
       ['<21', 'High', 'Male', 'Married', 'No'],
       ['21-35', 'High', 'Male', 'Single', 'Yes'],
       ['>35', 'Medium', 'Male', 'Single', 'Yes'],
       ['>35', 'Low', 'Female', 'Single', 'Yes'],
       ['>35', 'Low', 'Female', 'Married', 'No'],
       ['21-35', 'Low', 'Female', 'Married', 'Yes'],
       ['<21', 'Medium', 'Male', 'Single', 'No'],
       ['<21', 'Low', 'Female', 'Married', 'Yes'],
       ['>35', 'Medium', 'Female', 'Single', 'Yes'],
       ['<21', 'Medium', 'Female', 'Married', 'Yes'],
       ['21-35', 'Medium', 'Male', 'Married', 'Yes'],
       ['21-35', 'High', 'Female', 'Single', 'Yes'],
       ['>35', 'Medium', 'Male', 'Married', 'No']], dtype=object)

In [9]:
def unique_val(rows,col):
    return set(row[col] for row in rows)

def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label]=0
        counts[label]+=1
        
    return counts

{'No': 5, 'Yes': 9}

In [10]:
class Question:

    def __init__(self,column,value):
        self.column = column
        self.value = value
          
    def match(self,example):
        value = example[self.column]
        return value == self.value
    def __repr__(self):
        return ("Is %s == %s ?"%(header[self.column],self.value))
  


In [19]:
def partition(question,rows):
    true_rows,false_rows = [],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
            
    return true_rows,false_rows       

([array(['>35', 'Medium', 'Male', 'Single', 'Yes'], dtype=object),
  array(['<21', 'Medium', 'Male', 'Single', 'No'], dtype=object),
  array(['>35', 'Medium', 'Female', 'Single', 'Yes'], dtype=object),
  array(['<21', 'Medium', 'Female', 'Married', 'Yes'], dtype=object),
  array(['21-35', 'Medium', 'Male', 'Married', 'Yes'], dtype=object),
  array(['>35', 'Medium', 'Male', 'Married', 'No'], dtype=object)],
 [array(['<21', 'High', 'Male', 'Single', 'No'], dtype=object),
  array(['<21', 'High', 'Male', 'Married', 'No'], dtype=object),
  array(['21-35', 'High', 'Male', 'Single', 'Yes'], dtype=object),
  array(['>35', 'Low', 'Female', 'Single', 'Yes'], dtype=object),
  array(['>35', 'Low', 'Female', 'Married', 'No'], dtype=object),
  array(['21-35', 'Low', 'Female', 'Married', 'Yes'], dtype=object),
  array(['<21', 'Low', 'Female', 'Married', 'Yes'], dtype=object),
  array(['21-35', 'High', 'Female', 'Single', 'Yes'], dtype=object)])

In [26]:
def gini(rows):
    impurity = 1
    counts = class_counts(rows)
    for label in counts:
        pr = counts[label]/float(len(rows))
        impurity -= pr**2
        
    return impurity

def info_gain(left,right,curr_uncertainty):
    p = float(len(left))/(len(left)+len(right))
    return curr_uncertainty - p*gini(left) - (1-p)*gini(right)


0.000850340136054395

In [27]:
class Leaf:
      def __init__(self,rows):
            self.predictions = class_counts(rows)
            
class Decision_Node:
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        

In [56]:
def find_best_split(rows):
    best_gain=0
    best_question = None
    n_features = len(rows[0])-1
    curr_uncertainty = gini(rows)
    for i in range(n_features):
        values = set([row[i] for row in rows])
        for val in values:
            question = Question(i,val)
            true_rows,false_rows = partition(question,rows)
            if len(true_rows)==0 or len(false_rows)==0:
                continue
            infogain = info_gain(true_rows,false_rows,curr_uncertainty)
            if infogain >= best_gain:
                best_gain = infogain
                best_question = question
               
    return best_gain,best_question
    
    
    
    return best_gain,best_question

In [57]:
find_best_split(training_data)

(0.10204081632653056, Is Age == 21-35 ?)

In [58]:
def build_tree(rows):
    gain,question = find_best_split(rows)
    
    if gain==0:
        return Leaf(rows)
    true_rows,false_rows = partition(question,rows)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question,true_branch,false_branch)
    

In [59]:
def print_tree(node):
    if isinstance(node,Leaf):
        print(node.predictions)
        return
    
    print("Split occurs at: "+ str(node.question))
    
    print("--->True")
    print_tree(node.true_branch)
    
    print("--->False")
    print_tree(node.false_branch)
    
    

In [60]:
def classify(Node,row):
    if isinstance(Node,Leaf):
        return Node.predictions
    if Node.question.match(row):
        return classify(Node.true_branch,row)
    else:
        return classify(Node.false_branch,row)

In [61]:
Node = build_tree(training_data)
print_tree(Node)
row = ['<21','Low','Female','Married']
classify(Node,row)

Split occurs at: Is Age == 21-35 ?
--->True
{'Yes': 4}
--->False
Split occurs at: Is Gender == Male ?
--->True
Split occurs at: Is Age == >35 ?
--->True
Split occurs at: Is Marital Status == Single ?
--->True
{'Yes': 1}
--->False
{'No': 1}
--->False
{'No': 3}
--->False
Split occurs at: Is Marital Status == Single ?
--->True
{'Yes': 2}
--->False
Split occurs at: Is Age == >35 ?
--->True
{'No': 1}
--->False
{'Yes': 2}


{'Yes': 2}